In [1]:
from typing import List

import pandas as pd
import numpy as np
import pglast

import logparsing
import action_selection
import action_generation.index_actions as index_actions
import db_connector

In [2]:
[str(a) for a in index_actions.DropIndexGenerator(db_connector.get_existing_indexes())]

['DROP INDEX IF EXISTS idx_jungle_intfield3_floatfield7;',
 'DROP INDEX IF EXISTS idx_jungle_intfield7;',
 'DROP INDEX IF EXISTS idx_jungle_uuidfield;']

In [3]:
workload_csv = "artifacts/workload/workload.csv"
timeout = 100

In [4]:
col_mappings = db_connector.get_col_mappings()
parsed = logparsing.parse_csv_log(workload_csv)
filtered = logparsing.aggregate_templates(parsed, col_mappings, 0.01)
colrefs = action_selection.get_workload_colrefs(filtered)

In [19]:
action_selection.generate_sql(workload_csv)

cost estimate iter: 0
cost estimate iter: 0
cost estimate iter: 0
cost estimate iter: 0
cost estimate iter: 0


In [26]:
db_connector.get_existing_indexes()

[('jungle',
  'idx_jungle_intfield3_floatfield7',
  'CREATE INDEX idx_jungle_intfield3_floatfield7 ON public.jungle USING btree (int_field3, float_field7)',
  ['int_field3', 'float_field7']),
 ('jungle',
  'idx_jungle_intfield7',
  'CREATE INDEX idx_jungle_intfield7 ON public.jungle USING btree (int_field7)',
  ['int_field7']),
 ('jungle',
  'idx_jungle_uuidfield',
  'CREATE INDEX idx_jungle_uuidfield ON public.jungle USING btree (uuid_field)',
  ['uuid_field'])]

In [10]:
gen = index_actions.ExhaustiveIndexGenerator(colrefs,2)

In [11]:
[str(a) for a in gen]

['CREATE INDEX IF NOT EXISTS idx_jungle_intfield3 ON jungle (int_field3);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_intfield7 ON jungle (int_field7);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_uuidfield ON jungle (uuid_field);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield1 ON jungle (float_field1);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield4 ON jungle (float_field4);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield3 ON jungle (float_field3);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield8 ON jungle (float_field8);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield0 ON jungle (float_field0);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield2 ON jungle (float_field2);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield9 ON jungle (float_field9);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield5 ON jungle (float_field5);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield6 ON jungle (float_field6);',
 'CREATE INDEX IF NOT EXISTS idx_jungle_floatfield7 

In [3]:

    # Resulting hypopg indexes and corresponding CREATE INDEX action
    hypoconn = db_connector.get_conn()
    print("Estimating pre-hypothetical costs")
    before = action_selection.estimate_cost(parsed, hypoconn, iterations = 3)
    hypo_results = []

    # Install hypothetical indexes
    exhaustive = index_actions.ExhaustiveIndexGenerator(colrefs, 2)
    actions = list(exhaustive)
    for action in actions:
        sql_str = str(action)
        print(f"Creating hypoindex {sql_str}")
        hypo_sql = f"SELECT * FROM hypopg_create_index('{sql_str}') ;"
        hypo_results.append((action, hypoconn.execute(hypo_sql).fetchall()))

    print("Estimating workload cost with hypothetical indexes")
    hypo_ests = action_selection.estimate_cost(parsed, hypoconn, iterations = 3)

    hypo_ests['cost_diff'] = hypo_ests['cost'] - before['cost']
    hypo_results, hypo_ests

Estimating pre-hypothetical costs
cost estimate iter: 0
cost estimate iter: 1
cost estimate iter: 2
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_trust_sourceuid ON trust (source_u_id);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_trust_targetuid ON trust (target_u_id);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_trust_sourceuid_targetuid ON trust (source_u_id, target_u_id);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_trust_targetuid_sourceuid ON trust (target_u_id, source_u_id);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_review_iid ON review (i_id);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_review_uid ON review (u_id);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_review_iid_uid ON review (i_id, u_id);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_review_uid_iid ON review (u_id, i_id);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_item_iid ON item (i_id);
Creating hypoindex CREATE INDEX IF NOT EXISTS idx_useracct_uid ON useracct (u_id);
